<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install morfeusz2

     |████████████████████████████████| 8.5 MB 7.5 MB/s 


In [3]:
# instalacja morfeusza
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2


# instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz' pl_spacy_model_morfeusz-0.1.3.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

--2022-01-26 20:13:45--  http://download.sgjp.pl/apt/sgjp.gpg.key
Resolving download.sgjp.pl (download.sgjp.pl)... 193.0.67.154
Connecting to download.sgjp.pl (download.sgjp.pl)|193.0.67.154|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029 (1.0K) [application/pgp-keys]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.00K  --.-KB/s    in 0s      

2022-01-26 20:13:45 (138 MB/s) - written to stdout [1029/1029]

OK
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/comput

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.utils import simple_preprocess
from collections import Counter
import gc
import morfeusz2
import spacy
from spacy.lang.pl.stop_words import STOP_WORDS

In [5]:
# Wyświetlanie szerszych kolumn, aby nie skracać wyświetlanych nazw tematów
pd.set_option('max_colwidth', 400)

In [6]:
morf = morfeusz2.Morfeusz()

In [7]:
df = pd.read_excel('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/budzet_obywatelski.xlsx', sheet_name='Arkusz1')
df.rename(columns={'rok ': 'rok', 'miasto ':'miasto'}, inplace=True)
df = df[~df['nazwa'].isna()]
df['miasto'] = df['miasto'].map(lambda x: x.upper())
# Usunięcie pustych wpisów

# Indeksy do usunięcia
rem_index = df[df['nazwa'].isnull()].index.tolist()
rem_index += df[df['nazwa']=='bd'].index.tolist()

df.drop(df.index[rem_index], inplace=True)
# Usunięcie duplikatów
df = df.drop_duplicates()
df['nazwa'] = df['nazwa'].apply(str)
#usuniecie kolumny wszystko
if 'WSZYSTKO' in df: del df['WSZYSTKO']
df.sample(10)

,rok,miasto,nazwa
12421,2018/19,ŁÓDŹ,Kolorowy ogród PM 159 - ul. Łączna 53.
19474,2018,WARSZAWA,Mini park na Wrzecionie 7
22912,2020,WARSZAWA,Aktywne przejście na ul. Chrościckiego
995,2020,GDAŃSK,MONITOROWANIE JAKOŚCI POWIETRZA W DZIELNICY OSOWA
22624,2020,WARSZAWA,Treningi sportowe - spełnienie marzeń dzieci
12951,2018/19,ŁÓDŹ,"Przebudowa skrzyżowania ulic : Tatarkiewicza, Ananasowej, Oliwkowej w Osiedlu Nowosolna."
6631,2019,KALISZ,Zajęcia gimnastyczne dla seniorów 60+.
2221,2018,BIELSKO-BIAŁA,Rozbudowa placu zabaw przy ul. Mazańcowickiej 34
8836,2018,WROCŁAW,Oaza nad Kasiną II
3615,2020,KATOWICE,Integracja i aktywizacja środowiska lokalnego JP nr 5


In [8]:
df.loc[(df['miasto'] == 'WARSZAWA'), "czy_WAW"] = 1
df.loc[df["czy_WAW"].isnull(),  "czy_WAW"] = 0
df["czy_WAW"].value_counts()

0.0    18431
1.0     4599
Name: czy_WAW, dtype: int64

# WARSZAWA

In [9]:
df_waw = df[df['miasto'] == 'WARSZAWA']
df_waw.head()

,rok,miasto,nazwa,czy_WAW
18418,2018,WARSZAWA,Domki dla jeży,1.0
18419,2018,WARSZAWA,Wolskie lato filmowe,1.0
18420,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,1.0
18421,2018,WARSZAWA,Czysta Wisła na Pradze Południe,1.0
18422,2018,WARSZAWA,Wola na 2 koła! Kontraruch rowerowy w całej dzielnicy,1.0


In [10]:
data_waw =[]

In [11]:
for rows in range(1,df_waw.shape[0]):
  text = df_waw.iloc[rows, 2]
  miasto = df_waw.iloc[rows, 1]
  rok = df_waw.iloc[rows, 0]
  print(text)
  analysis = morf.analyse(text)
  for interpretation in analysis:
      start_ind, end_ind, interp = interpretation
      form, lemma, tag, qualifiers, _ = interpretation[2]
      data_waw.append((rok, miasto, text, start_ind, end_ind, form,lemma,tag,qualifiers,_))

Wolskie lato filmowe
Chronimy jerzyki i wróble na Woli - skrzynki lęgowe
Czysta Wisła na Pradze Południe
Wola na 2 koła! Kontraruch rowerowy w całej dzielnicy
Bielańska plaża nad Wisłą
Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe
leczenie i sterylizacja wolno żyjących kotów
Ćwiczenia dla seniorów na otwartych siłowniach
Rozwijamy wolskie maluchy
Skarby Woli. Czego nie wiesz o swojej dzielnicy?
Chronimy jerzyki i wróble na Bielanach - skrzynki lęgowe
Czyste powietrze dla Targówka - nasadzenia drzew
Tai Chi na świeżym powietrzu
Ogródek przy przystanku autobusowym "Antka Rozpylacza 01"
Wiem czym oddycham! - tablica informująca o poziomie zanieczyszczenia powietrza (Praga-Południe)
Virtualna Warszawa – turystyka dostępna dla wszystkich, również niepełnosprawnych.
Ratujemy drzewa Parku Skaryszewskiego - wiązania elastyczne w koronach drzew na podstawie ekspertyzy dendrologicznej.
Serek - Miejsce spotkań i relaksu przy metrze Słodowiec
Wodny plac zabaw dla Białołęki
Nakarm

In [12]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
df_waw_morf4 = pd.DataFrame(data_waw, columns=cols)
df_waw_morf4.head(40)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:acc:m2.m3.f.n:pos,[],[]
1,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
2,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:acc:n:pos,[],[]
3,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:nom.voc:n:pos,[],[]
4,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolskie,subst:pl:nom.acc.voc:n:pt,[nazwa_geograficzna],[]
5,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolska,subst:pl:nom.acc.voc:f,[nazwisko],[]
6,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolski,depr:pl:nom.acc.voc:m2,[nazwisko],[]
7,2018,WARSZAWA,Wolskie lato filmowe,1,2,lato,lato,subst:sg:nom.acc.voc:n:ncol,[nazwa_pospolita],[]
8,2018,WARSZAWA,Wolskie lato filmowe,2,3,filmowe,filmowy,adj:pl:acc:m2.m3.f.n:pos,[],[]
9,2018,WARSZAWA,Wolskie lato filmowe,2,3,filmowe,filmowy,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]


In [13]:
df_waw_morf4.shape

(89425, 10)

In [50]:
df_waw_morf4['qualifiers'].map(lambda x: str(x)).value_counts()

[]                                                             47582
['nazwa_pospolita']                                            30221
['nazwisko']                                                    5890
['nazwa_geograficzna']                                          3405
['nazwa_geograficzna', 'nazwisko']                              1174
['imię']                                                         437
['człon_nazwy_geograficznej']                                    224
['nazwa_organizacji']                                            166
['imię', 'nazwisko']                                             137
['nazwa_firmy']                                                   35
['imię', 'nazwa_geograficzna']                                    28
['człon_nazwy_geograficznej', 'nazwisko']                         19
['nazwa pospolita']                                               19
['nazwa_instytucji']                                              18
['człon_nazwy_organizacji']       

In [52]:
df_waw_morf4['_'].map(lambda x: str(x)).value_counts().head(60)

[]                          85243
['daw.']                      842
['pot.']                      638
['przest.']                   519
['hom.']                      408
['muz.']                      382
['zool.']                     144
['książk.']                   142
['gwar.']                     108
['jęz.']                       97
['rzad.']                      88
['sport.']                     80
['bot.']                       73
['pogard.']                    37
['środ.']                      36
['z_d.']                       32
['arch.,char.']                32
['anat.']                      30
['karc.']                      26
['wojsk.']                     22
['daw.,praw.']                 21
['szkol.']                     19
['wulg.']                      18
['przest.,techn.']             18
['hist.']                      17
['pot.,środ.']                 16
['komp.']                      15
['przen.']                     15
['pot.,bot.']                  14
['char.']     

In [15]:
czasownik = ['fin', 'bedzie', 'aglt', 'praet', 'impt', 'imps', 'inf', 'pcon', 'pant', 'ger', 'pact', 'ppas']
df_waw_morf4_verb = df_waw_morf4[df_waw_morf4['tag'].str.contains('|'.join(czasownik))]

In [16]:
df_waw_morf4_verb.lemma.value_counts().head(40)

zająć           365
zabawić         216
spotkać         185
przejść         148
oświetlić       127
zielenić        111
wzdłużyć        102
zakupić          99
nasadzić         93
być              61
żyć              46
doświetlić       46
skrzyżować       46
chronić          41
urządzić         35
programować      34
pić              34
otworzyć         34
szkolić          32
dojść            27
ratować          27
woleć            26
śmiecić          26
doposażyć        26
ćwiczyć          26
rozwijać         25
przystać:Vp      24
przestrzenić     23
mieć             22
grać             21
parkować         21
czytać           20
brakować         19
wesprzeć         19
zwalniać         19
zjednoczyć       18
wyposażyć        18
posadzić         18
pływać           16
chcieć           16
Name: lemma, dtype: int64

In [17]:
df_waw_morf4[df_waw_morf4['_'].map(set(['zool.']).issubset)]

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
13,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
17,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,3,4,wróble,wróbel,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
104,2018,WARSZAWA,Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
108,2018,WARSZAWA,Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe,3,4,wróble,wróbel,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
215,2018,WARSZAWA,Chronimy jerzyki i wróble na Bielanach - skrzynki lęgowe,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
...,...,...,...,...,...,...,...,...,...,...
86153,2020,WARSZAWA,Muzyka nad Stawami Cietrzewia,3,4,Cietrzewia,cietrzew,subst:sg:gen.acc:m2,[nazwa_pospolita],[zool.]
88095,2020,WARSZAWA,"„Chronimy ptaki, nietoperze i wiewiórki - zawieszenie skrzynek lęgowych oraz wykonanie tratw dla ptactwa wodnego”",4,5,nietoperze,nietoperz,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]
88360,2020,WARSZAWA,Wieże lęgowe dla jerzyków,3,4,jerzyków,jerzyk,subst:pl:gen:m2,[nazwa_pospolita],[zool.]
88420,2020,WARSZAWA,Chronimy jerzyki i wróble na Żoliborzu - montaż skrzynek lęgowych,1,2,jerzyki,jerzyk,subst:pl:nom.acc.voc:m2,[nazwa_pospolita],[zool.]


In [18]:
df_waw_morf4.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_WAW.csv', index=False)

## Reszta miast

In [19]:
df_roc = df[df['miasto'] != "WARSZAWA"]
df_roc.head(10)

,rok,miasto,nazwa,czy_WAW
0,2020,GDYNIA,Projektistatus,0.0
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0.0
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację,0.0
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",0.0
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną,0.0
5,2020,GDYNIA,Cotygodniowe weekendowe lekcje tańca towarzyskiego dla dorosłych,0.0
6,2020,GDYNIA,Meble miejskie zaprojektowane przez dzieci. Podziemia Hal Targowych w Gdyni.,0.0
7,2020,GDYNIA,"Odnowienie podwórek w obrębie ulic Płk. Dąbka, Kwiatkowskiego, Sucharskiego, Ledóchowskiego i Maciejewicza",0.0
8,2020,GDYNIA,Poprawa atrakcyjności skweru przy skrzyżowaniu Wielkopolska-Łowicka-Płocka przez nasadzenie roślin ozdobnych,0.0
9,2020,GDYNIA,Radarowe wyświetlacze prędkości na ul. Morskiej,0.0


In [20]:
data_roc2 =[]

In [21]:
for rows in range(1,df_roc.shape[0]):
  text = df_roc.iloc[rows, 2]
  miasto = df_roc.iloc[rows, 1]
  rok = df_roc.iloc[rows, 0]
  print(text)
  analysis = morf.analyse(text)
  for interpretation in analysis:
      start_ind, end_ind, interp = interpretation
      form, lemma, tag, qualifiers, _ = interpretation[2]
      data_roc2.append((rok, miasto, text, start_ind, end_ind, form,lemma,tag,qualifiers,_))

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Parkowe akcje animacje na Stefku! 
Ping-pong dla młodych. 
RADOSNY PLAC ZABAW- remont i doposażenie placu zabaw przy SP 109 im. Ludwiki Wawrzyńskiej. 
Plac zabaw Lecznicza-Łukasińskiego. 
Piaseczna – nie Bagnista. 
Ażury wzdłuż Podmiejskiej – Park Lecznicza. 
PRACOWNIA TECHNICZNA ... CIĄG DALSZY-doposażenie istniejącej pracowni w SP 109 im. Ludwiki Wawrzyńskiej. 
Grill w Parku Dąbrowskiego. 
Rzgowska 56 - usiądź odpocznij sobie. 
KOLOROWE SZATNIE - remont szatni dla uczniów przy SP 109 im. L. Wawrzyńskiej. 
Nowoczesna szkoła XXI wieku - odnowienie sal lekcyjnych w SP 109 im. L. Wawrzyńskiej. 
„Bezpieczeństwo w szkole i wokół niej" - wymiana ogrodzenia, modernizacja chodników, budowa ciągów komunikacyjnych, modernizacja szatni. 
Przedszkole pod Żyrafą zaprasza do ogrodowej dżungli. 
Remont podjazdu dla wózków dziecięcych i schodów prowadzących do wejścia. 
Zakup książek dla czytelników Filii nr 71 Biblioteki Miejskiej przy

In [22]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
df_roc_morf5 = pd.DataFrame(data_roc2, columns=cols)
df_roc_morf5.head(40)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,[nazwa_pospolita],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,[nazwa_pospolita],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,[nazwa_pospolita],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]
5,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:inst,[],[]
6,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",3,4,Estakadą,estakada,subst:sg:inst:f,[nazwa_pospolita],[]
7,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",4,5,Kwiatkowskiego,Kwiatkowski:Sm1,subst:sg:gen.acc:m1,[nazwisko],[]
8,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:C,conj,[],[]
9,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:I,interj,[],[]


In [23]:
df_roc_morf5.shape

(419564, 10)

In [24]:
df_roc_morf5[df_roc_morf5['_'].map(set(['zool.']).issubset)].sample(60)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
114618,2018,KATOWICE,L4/7/V - Rower miejski dla Os. Paderewskiego,9,10,Os,osa,subst:pl:gen:f,[nazwa_pospolita],[zool.]
301788,2018/19,ŁÓDŹ,Ratujmy zabytkową studnię kręgową z żurawiem na ul. Beskidzkiej.,5,6,żurawiem,żuraw:Sm2,subst:sg:inst:m2,[nazwa_pospolita],[zool.]
371444,2019,KRAKÓW,TAK DLA WILGI,2,3,WILGI,wilga,subst:sg:gen:f,[nazwa_pospolita],[zool.]
233096,2020,WROCŁAW,"Naprawa ogrodzenia wokół MOPS przy ul. Ciepłej i dodanie oświetlenia wew. osiedla, które zwiększy bezpieczeństwo",3,4,MOPS,mops,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
123925,2020,PIEKARY,10. Budki lęgowe dla ptaków jerzyków – naturalny sposób na komary,6,7,jerzyków,jerzyk,subst:pl:gen:m2,[nazwa_pospolita],[zool.]
82771,2020,GLIWICE,Wykonanie sygnalizacji ostrzegawczej na 3 przejsciach dla pieszych: W rejonie Biedronki oraz na skrzyzowaniach ul. Strzelców Bytomskich Z ul. Chatka Puchatka i ul. Radosna.,11,12,Biedronki,biedronka,subst:sg:gen:f,[nazwa_pospolita],[zool.]
72696,2018,CZĘSTOCHOWA,Wieze legowe dla Jerzyków,3,4,Jerzyków,jerzyk,subst:pl:gen:m2,[nazwa_pospolita],[zool.]
61037,2019,BYTOM,"Otwarta Strefa Aktywnosci - silownia W Karbiu przy MDK nr 2"" - utworzenie otwartej silowni, szesé stanowisk wyposazonych W jak najwiecej róznorodnych i odpornych na wandalizm przyrzadów do éwiczen na powietrzu",21,22,jak,jak:S,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
134022,2020,SIEMIANOWICE ŚLĄSKIE,7. Wymiana nawierzchni drogi dojazdowej przy ul. Michałkowickiej 25-35 wraz z utworzeniem miejsc postojowych wzdłuż odcinka drogi jak wyżej oraz utwardzenie dojazdu do wielostanowiskowego szeregu garaży zlokalizowanych przy ul. Michałkowickiej w Siemianowicach Śląskich,21,22,jak,jak:S,subst:sg:nom:m2,[nazwa_pospolita],[zool.]
146146,2020,TYCHY,Piknik rodzinny na os. B,3,4,os,osa,subst:pl:gen:f,[nazwa_pospolita],[zool.]


In [25]:
df_roc_morf5.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_ROC.csv', index=False)

# 2018

In [26]:
df_2018 = df[df['rok'] == 2018]
df_2018.head(10)

,rok,miasto,nazwa,czy_WAW
654,2018,GDYNIA,Urzadzenie zieleni miejskiej miedzy ul. Dedala 6 a korona klifu,0.0
655,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0.0
656,2018,GDYNIA,Montaz fontann wody pitnej W ruchliwych punktach dzielnicy: przy placu zabaw oraz na poczatku sciezki rowerowej,0.0
657,2018,GDYNIA,"Rozbudowa placu zabaw dla dzieci mtodszych, starszych i mtodziezy przy ulicy Dedala 6-8",0.0
658,2018,GDYNIA,Montaz dwóch defibrylatorów - przy parafii przy ul. Rybaków oraz przy wspólnocie mieszkaniowej przy ul. Ikara 6,0.0
659,2018,GDYNIA,"Nasadzenia kwiatów i innych kompozycji roslinnych przy ulicach Dedala, Ikara, Zielonej i Rybaków",0.0
660,2018,GDYNIA,Oswietlenie boiska przy Szkole Podstawowej nr 28,0.0
661,2018,GDYNIA,Wykonanie ogólnodostepnego parkingu na terenie parafii Biskupa Józefa Sebastiana Pelczara,0.0
662,2018,GDYNIA,"Budowa rekreacyjnych torów do jazdy typu pumptrack wraz Z uzupelniajaca zielenia i miejscami do odpoczynku W poblizu przystanku autobusowego ""Gierdziejewskiego""",0.0
663,2018,GDYNIA,"Wymiana oswietlenia przy gtównych ulicach dzielnicy na oprawy LED oraz montaz solarnych tawek (ul. Okreina, Chwarznienska, Wiczlinska, Zaruskiego, Filipkowskiego i Staniszewskiego)",0.0


In [27]:
data_2018 =[]

In [28]:
def morfeusz_create_morf_table(df, data):
  for rows in range(1,df.shape[0]):
    text = df.iloc[rows, 2]
    miasto = df.iloc[rows, 1]
    rok = df.iloc[rows, 0]
    #print(text)
    analysis = morf.analyse(text)
    for interpretation in analysis:
        start_ind, end_ind, interp = interpretation
        form, lemma, tag, qualifiers, _ = interpretation[2]
        data.append((rok, miasto, text, start_ind, end_ind, form,lemma,tag,qualifiers,_))
  return data

In [29]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
list_morf = morfeusz_create_morf_table(df_2018, data_2018)
df_2018_morf = pd.DataFrame(list_morf, columns=cols)
df_2018_morf.head(40)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0,1,Zagospodarowanie,zagospodarować,ger:sg:nom.acc:n:perf:aff,[],[]
1,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",1,2,terenu,teren,subst:sg:gen:m3,[nazwa_pospolita],[]
2,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:acc,[],[]
3,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:dat,[],[]
4,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:loc,[],[]
5,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",3,4,bytym,bytym,ign,[],[]
6,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",4,5,baraku,barak,subst:sg:gen:m3,[nazwa_pospolita],[]
7,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",4,5,baraku,barak,subst:sg:loc:m3,[nazwa_pospolita],[]
8,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",4,5,baraku,barak,subst:sg:voc:m3,[nazwa_pospolita],[]
9,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",5,6,przy,przy,prep:loc,[],[]


In [30]:
df_2018_morf.shape

(122473, 10)

In [31]:
df_2018_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2018.csv', index=False)

## 2019

In [32]:
df_2019 = df[df['rok'] == 2019]
df_2019.head(10)

,rok,miasto,nazwa,czy_WAW
360,2019,GDYNIA,"Zakup nowoczesnego, W petni wyposazonego ambulansu ratunkowego Z zabudowq medycznq, zgodnego Z wymogami Ministerstwa Zdrowia, dla Miejskiej Stacji Pogotowia Ratunkowego W Gdyni",0.0
361,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0.0
362,2019,GDYNIA,"Ogrodzony wybieg dla psów, W poblizu schroniska dla zwierzgt Ciapkowo"" przy ulicach towicka - Olkuska W dzielnicy Maty Kack, dla psów Z catej Gdyni",0.0
363,2019,GDYNIA,Kampania edukacyjna dot. segregacji odpadów oraz mieciomaty/recyklomaty - urzadzenia do zdawania odpadów aluminiowych i szkta Z wymianq na bilety/znizki Z uzyciem Karty Mieszkanca,0.0
364,2019,GDYNIA,"Gdynskie taki kwietne zlokalizowane w 9 dzielnicach: Dqbrowa, Sródmiescie, Karwiny, Obtuze, Cisowa, Chylonia, Wielki Kack, Pogórze oraz Ortowo",0.0
365,2019,GDYNIA,Dostosowanie Estakady Kwiatkowskiego do ruchu rowerowego na odcinku Morska- Hutnicza,0.0
366,2019,GDYNIA,Mobilna stacja pomiarowa powietrza i obstugi interwencji zwiqzanych Z porzqdkiem publicznym (dron) oraz kampania informacyjna dot. zanieczyszczania powietrza,0.0
367,2019,GDYNIA,Organizacji pórkolonii letnich dla dzieci na terenie 23 szkót podstawowych W 21 dzielnicach Gdyni,0.0
368,2019,GDYNIA,"Ogrody deszczowe na petlach autobusowych/trolejbusowych (Chabrowa, Grabówek SKM, Obtuze Maciejewicza, Pogórze (ul. Czernickiego), Dqbrowa Mietowa, Karwiny Tesco, Ortowo SKM Klif)",0.0
369,2019,GDYNIA,Transport i wsparcie dla seniorów (65+) W dotarciu do miejsc uzytecznosci publicznej,0.0


In [33]:
data_2019 =[]

In [34]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
list_morf = morfeusz_create_morf_table(df_2019, data_2019)
df_2019_morf = pd.DataFrame(list_morf, columns=cols)
df_2019_morf.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,park,subst:pl:nom.acc.voc:m3,[nazwa_pospolita],[]
1,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:sg:gen:f,[nazwa_pospolita],[]
2,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:pl:nom.acc.voc:f,[nazwa_pospolita],[]
3,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:sg:gen:f,"[imię, nazwisko]",[]
4,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:pl:nom.acc.voc:f,"[imię, nazwisko]",[]
5,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sm1,subst:sg:gen:m1,[nazwisko],[]
6,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sm1,depr:pl:nom.acc.voc:m2,[nazwisko],[]
7,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",1,2,kieszonkowe,kieszonkowy,adj:pl:acc:m2.m3.f.n:pos,[],[]
8,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",1,2,kieszonkowe,kieszonkowy,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
9,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",1,2,kieszonkowe,kieszonkowy,adj:sg:acc:n:pos,[],[]


In [35]:
df_2019_morf.shape

(176322, 10)

In [36]:
df_2019_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2019.csv', index=False)

## 2020

In [37]:
df_2020 = df[df['rok'] == 2020]
df_2020.head(10)

,rok,miasto,nazwa,czy_WAW
0,2020,GDYNIA,Projektistatus,0.0
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0.0
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację,0.0
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",0.0
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną,0.0
5,2020,GDYNIA,Cotygodniowe weekendowe lekcje tańca towarzyskiego dla dorosłych,0.0
6,2020,GDYNIA,Meble miejskie zaprojektowane przez dzieci. Podziemia Hal Targowych w Gdyni.,0.0
7,2020,GDYNIA,"Odnowienie podwórek w obrębie ulic Płk. Dąbka, Kwiatkowskiego, Sucharskiego, Ledóchowskiego i Maciejewicza",0.0
8,2020,GDYNIA,Poprawa atrakcyjności skweru przy skrzyżowaniu Wielkopolska-Łowicka-Płocka przez nasadzenie roślin ozdobnych,0.0
9,2020,GDYNIA,Radarowe wyświetlacze prędkości na ul. Morskiej,0.0


In [38]:
data_2020 =[]

In [39]:
cols=['rok','miasto','nazwa','start_ind', 'end_ind', 'form','lemma','tag','qualifiers','_']
list_morf = morfeusz_create_morf_table(df_2020, data_2020)
df_2020_morf = pd.DataFrame(list_morf, columns=cols)
df_2020_morf.head(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,[nazwa_pospolita],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,[nazwa_pospolita],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,[nazwa_pospolita],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]
5,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:inst,[],[]
6,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",3,4,Estakadą,estakada,subst:sg:inst:f,[nazwa_pospolita],[]
7,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",4,5,Kwiatkowskiego,Kwiatkowski:Sm1,subst:sg:gen.acc:m1,[nazwisko],[]
8,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:C,conj,[],[]
9,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",5,6,a,a:I,interj,[],[]


In [40]:
df_2020_morf.shape

(183308, 10)

In [41]:
df_2020_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2020.csv', index=False)